# **Phân tích và trực quan hóa dữ liệu mạng xã hội**
### Mạng xã hội: *Facebook*
### Group: *UIT K11*
https://www.facebook.com/groups/UIT.K2016

## Thu thập dữ liệu và tiền xử lí

Đọc tất cả các user trong group UIT K11

In [0]:
import pandas as pd 
data = pd.read_csv("UIT.K11.csv")
data.head()

,Name,UID
0,Trần Công,100010630570591
1,Lắc Si Lê,100003265219762
2,Hoà Võ,100013778894026
3,Minh Quân,100003050081671
4,Truong Nguyen,100037769810406


In [0]:
list_uid_members = data["UID"].tolist()
print(type(list_uid_members[0]))

<class 'int'>


Chuyển giá trị sang kiểu string

In [0]:
list_uid_members_str = list(map(str,list_uid_members))
print(type(list_uid_members_str[0]))

<class 'str'>


Gọi tới api của Facebook để lấy bạn bè của tất cả user

In [0]:
import requests as req
token = 'EAAAAZAw4FxQIBAC0PsJZCjx8pu3Pvjq4ZAifAxicj6UJuhzrA98rwA1aW2RO6mHLYNKyMQy9AZA9SivEPjvMZBk9GZBw0UpDQ0IDdKrLNPwQPkSYvbiPcOs525IjDHGNcZCZCHkaBj2qqf3EkF8bouZAJdoQN42ZBJU5HedcJFZCO0eztH9qKsZAaujtX8mkAdZA8vgkZDs'

Lấy ra tất cả các user trong group đặt chế độ public friend trên Facebook. Các user không public friend coi như không nằm trong group

In [0]:
import requests as req
token = 'YOUR-TOKEN-HERE!'
list_user_public_friend = []
for uid in list_uid_members_str:
  try:
    res = req.get('https://graph.facebook.com/v4.0/'+ uid + '/friends?fields=id', params={'access_token': token})
    friends_of_user = res.json()['data']
    # Nếu friends_of_user != null => user này đặt chế độ public friend 
    if friends_of_user:
      list_user_public_friend.append(uid)
  except:
    continue

In [0]:
print(list_user_public_friend[0])

100010630570591


In [0]:
print(len(list_user_public_friend))

1431


Định dạnh trả về của api

In [0]:
res = req.get('https://graph.facebook.com/v4.0/100009157116574/friends?fields=id', params={'access_token': token})
print(res.json())

{'data': [{'id': '529163359'}, {'id': '608115349'}, {'id': '720083187'}, {'id': '1303000653'}, {'id': '1371819347'}, {'id': '1834012460'}, {'id': '100000139238255'}, {'id': '100000147657530'}, {'id': '100000170377497'}, {'id': '100000248454383'}, {'id': '100000487774478'}, {'id': '100000517419854'}, {'id': '100000601257708'}, {'id': '100000607404624'}, {'id': '100000757973562'}, {'id': '100000909431136'}, {'id': '100001006448357'}, {'id': '100001063881293'}, {'id': '100001215087238'}, {'id': '100001294330407'}, {'id': '100001296555549'}, {'id': '100001375618899'}, {'id': '100001649633212'}, {'id': '100001778275428'}, {'id': '100001805609023'}], 'paging': {'cursors': {'before': 'QVFIUmwyWFBrMUxLQ2k2Mzd2eW9ZAZA3lYS3N3cWJudUxtNVRINGNxV09oS0twOHNoX242NDcwNENIUE5fbElSZA25KVzcZD', 'after': 'QVFIUjJYYXBxQzlfYlYwRnRiLWl0Ym5yLW9lemMzVnFjd1EzcFBaRGZAPLWZApS1BBVGJ1NkdpUnQ2eHNBOVlZALUxIa1pLcGlrSGVSVVdEZA3NFYlhDY2p4NXh3'}, 'next': 'https://graph.facebook.com/v4.0/100009157116574/friends?access_toke

Với mỗi user chỉ chọn ra các bạn bè nằm trong group UIT K11. Đồng thời ghi vào file

In [0]:
import csv
with open('destinationestination_fillter.csv', mode='w+') as csvFile:
  writer = csv.writer(csvFile, delimiter=',')

  for uid in list_user_public_friend:
    res = req.get('https://graph.facebook.com/v4.0/'+ uid + '/friends?fields=id&&limit=5000', params={'access_token': token})
    friends_of_user = res.json()['data']
    friends_in_group = []
    for friend in friends_of_user:
      # Chỉ chọn ra những bạn bè nằm trong danh sách public friend
      if friend['id'] in list_user_public_friend:
        friends_in_group.append(friend['id'])
    row = [uid, *friends_in_group]
    writer.writerow(row)

csvFile.close()

## Tính toán các độ đo trong mạng

In [0]:
f = open("destination_fillter.csv", "r")
data= f.read()
file = open('destination_fillter.csv', 'r+');
data = file.read().replace(',,','').replace(',\n','\n')

# Loại bỏ đi phần tử cuối vì dòng cuối cũng kết thúc bằng kí hiệu '\n'
rows = data.split('\n')[:-1]

Chuyển thành mảng 2 chiều với mỗi phần tử trong mảng là 1 dòng trong dữ liệu

In [0]:
def convert_to_2D_array(rows):
    return [row.split(',') for row in rows]

In [0]:
array_2D = convert_to_2D_array(rows)
print(array_2D[17:19])

[['100033362014767', '100003392889872', '100004414514716'], ['100041592006552', '100005692004958', '100005854661259', '100011855953437']]


In [0]:
port networkx as nx
g = nx.Graph()

# Lấy phần tử [i][0] truyền vào làm nodes cho đồ thị.
for node in array_2D:
  g.add_node(node[0])

# Thêm cạnh cho đồ thị bằng cách bắt cặp phần tửi [i][0] với [i][j]
for item in array_2D:
  for index in range(1,len(item)):
    g.add_edge(item[0], item[index])

In [0]:
print(len(g.nodes()))

1441


In [0]:
print(len(g.edges()))

11197


Đọc file các uid và name thành dictionary để hiển thị tên lúc tính toán

In [0]:
import csv
reader = csv.reader(open('UIT.K11.csv', 'r'))
dict_user_uid = dict()

for row in reader:
  name, uid = row
  dict_user_uid[uid] = name
# Remove header
del dict_user_uid['UID']
print(next(iter(dict_user_uid.items())))

('100010630570591', 'Trần Công')


In [0]:
from prettytable import PrettyTable
# Hiển thị uid, tên và kết quả của độ đo
def findUser(dict_top10, dict_user_uid):
  t = PrettyTable(['UID','Name','Centrality'])
  for item in dict_top10:
    t.add_row([item, dict_user_uid[item], dict_top10[item]])
  print(t)

# Chọn ra 10 user có độ đo lớn nhất
def dict_top10(dict_centrality):
  arr_top10 = sorted(dict_centrality.items(), key=lambda kv: kv[1], reverse=True)[:10]
  dict_top10 = dict()
  for index, value in arr_top10:
    dict_top10[index] = value
  return dict_top10

**Degree Centrality**

In [0]:
dict_degree_centrality = dict( nx.degree_centrality(g))

dict_top10_degree = dict_top10(dict_degree_centrality)
print('Top 10 Key Player - Degree Centrality')
findUser(dict_top10_degree, dict_user_uid)

Top 10 Key Player - Degree Centrality
+-----------------+-----------------+---------------------+
|       UID       |       Name      |      Centrality     |
+-----------------+-----------------+---------------------+
| 100022371771809 |     Toàn Lê     | 0.18611111111111112 |
| 100027261426135 |     Bảo Phúc    | 0.17222222222222222 |
| 100012780131803 | Dương Đức Thành | 0.13194444444444445 |
| 100003782937571 |    Phương Lan   | 0.11875000000000001 |
| 100005101389366 |   Cường Nguyễn  | 0.10833333333333334 |
| 100007858844667 |      Võ Tân     | 0.10277777777777779 |
| 100028418463108 | Nguyễn Minh Kha | 0.10208333333333333 |
| 100005704122021 |  Nguyễn Thị Tâm | 0.10138888888888889 |
| 100009961336237 |  Nguyễn Hữu Thọ | 0.09791666666666667 |
| 100013232892681 |   Nguyễn Minh   | 0.09027777777777778 |
+-----------------+-----------------+---------------------+


**Betweenness Centrality**

In [0]:
dict_betweenness_centrality = dict( nx.betweenness_centrality(g))

dict_top10_betweenness = dict_top10(dict_betweenness_centrality)
print('Top 10 Key Player - Betweenness Centrality')
findUser(dict_top10_betweenness, dict_user_uid)

Top 10 Key Player - Betweenness Centrality
+-----------------+----------------------+----------------------+
|       UID       |         Name         |      Centrality      |
+-----------------+----------------------+----------------------+
| 100022371771809 |       Toàn Lê        | 0.08112260790619155  |
| 100027261426135 |       Bảo Phúc       | 0.06382553393735739  |
| 100012780131803 |   Dương Đức Thành    | 0.053292577294799304 |
| 100005101389366 |     Cường Nguyễn     | 0.03580512886646365  |
| 100003782937571 |      Phương Lan      | 0.03211173138494305  |
| 100009961336237 |    Nguyễn Hữu Thọ    | 0.028198727497095916 |
| 100005704122021 |    Nguyễn Thị Tâm    | 0.023319221601094908 |
| 100013728429775 |     Nguyễn Khang     | 0.019770269620545282 |
| 100003173570867 | Ngọc Anh Nguyễn Trần | 0.018512210207392858 |
|    1720796306   |       Tuấn Anh       | 0.01829668821326984  |
+-----------------+----------------------+----------------------+


**Closeness Centrality**

In [0]:
dict_closeness_centrality = dict( nx.closeness_centrality(g))

dict_top10_closeness = dict_top10(dict_closeness_centrality)
print('Top 10 Key Player - Centrality Centrality')
findUser(dict_top10_closeness,dict_user_uid)

Top 10 Key Player - Centrality Centrality
+-----------------+-----------------+---------------------+
|       UID       |       Name      |      Centrality     |
+-----------------+-----------------+---------------------+
| 100022371771809 |     Toàn Lê     | 0.44288548752834467 |
| 100027261426135 |     Bảo Phúc    | 0.44019044399368945 |
| 100012780131803 | Dương Đức Thành |  0.4149405141278946 |
| 100003782937571 |    Phương Lan   | 0.40776754770554097 |
| 100005704122021 |  Nguyễn Thị Tâm |  0.4063930503537245 |
| 100028418463108 | Nguyễn Minh Kha |  0.4060888639387891 |
| 100005101389366 |   Cường Nguyễn  | 0.40578513255214826 |
| 100007858844667 |      Võ Tân     | 0.40457473692932305 |
| 100013232892681 |   Nguyễn Minh   |  0.4009864909255153 |
| 100004674664188 |  Trần Hoàn Phúc |  0.3968798260586849 |
+-----------------+-----------------+---------------------+


**Eigenvector Centrality**

In [0]:
dict_eigenvector_centrality = dict(nx.eigenvector_centrality(g))

dict_top10_eigenvector = dict_top10(dict_eigenvector_centrality)
print('Top 10 Key Player - Eigenvector Centrality')
findUser(dict_top10_eigenvector,dict_user_uid)

Top 10 Key Player - Eigenvector Centrality
+-----------------+-----------------+---------------------+
|       UID       |       Name      |      Centrality     |
+-----------------+-----------------+---------------------+
| 100027261426135 |     Bảo Phúc    | 0.20686464555867493 |
| 100022371771809 |     Toàn Lê     | 0.20347673479521458 |
| 100028418463108 | Nguyễn Minh Kha | 0.16567443150055888 |
| 100003782937571 |    Phương Lan   |  0.1582691117661647 |
| 100007858844667 |      Võ Tân     | 0.15536095041182596 |
| 100005704122021 |  Nguyễn Thị Tâm | 0.14627100254898817 |
| 100013232892681 |   Nguyễn Minh   | 0.13897445020909105 |
| 100012780131803 | Dương Đức Thành | 0.13446323114404088 |
| 100005101389366 |   Cường Nguyễn  | 0.12678700888806088 |
| 100008574078227 |    Như Trương   | 0.12010892270463167 |
+-----------------+-----------------+---------------------+


## Trực quan hóa dữ liệu

In [0]:
!pip install python-igraph

     |████████████████████████████████| 378kB 2.8MB/s 
  Created wheel for python-igraph: filename=python_igraph-0.7.1.post6-cp36-cp36m-linux_x86_64.whl size=2218319 sha256=697ed4e2c55c1953e16719efded760631c80c34b7a566dada6222b6d0dbde49e
  Stored in directory: /root/.cache/pip/wheels/41/d6/02/34eebae97e25f5b87d60f4c0687e00523e3f244fa41bc3f4a7
Successfully built python-igraph


Danh sách tất cả các node trong đồ thị

In [0]:
# Danh sách node trong mạng
nodes = [item[0] for item in array_2D]
print(nodes[0])
# Output: 100010630570591

100010630570591


Tạo danh sách cạnh có dạnh index của các node <br>
Ex: ('100010630570591','100003265219762') -> (0,819)

In [0]:
# Danh sách cạnh trong mạng ()
edges = []
for i in range(len(array_2D)):
  for item in array_2D[i][1:]:
    edges.append((i, nodes.index(item)))
print(edges[0])
# Output: (0, 819)

(0, 819)


In [0]:
number_node = len(nodes)
print(number_node)
# Output: 1441 => node

1441


In [0]:
number_edge = len(edges)
print(number_edge)
# Output: 22394 => edges

22394


Tạo đồ thị

In [0]:
import igraph as ig
# Tạo đồ thị từ danh sách cạnh 
G=ig.Graph(edges, directed=False)

Tạo vị trí cho các nodes chế độ Kamada-Kawai 3 chiều

In [0]:
# Chia layout 3 chiều 
layt=G.layout('kk', dim=3)

In [0]:
print(type(layt))
print(layt[1])
print(layt)

<class 'igraph.layout.Layout'>
[-12.729917079025645, 3.1362231544865757, -17.040066077646934]
<Layout with 1441 vertices and 3 dimensions>


Chuyển tọa độ từ [(x1,y1,z1),(x2,y2,z2),...] => X = [x1,x2,...] , Y = [...]

In [0]:
# Chuyển tọa độ nodes
Xn=[layt[k][0] for k in range(number_node)]# tọa độ theo trục x của node
Yn=[layt[k][1] for k in range(number_node)]# tọa độ theo trục y
Zn=[layt[k][2] for k in range(number_node)]# tọa độ theo trục z
# Chuyển tọa độ cạnh
Xe=[]
Ye=[]
Ze=[]
for e in edges:
  Xe+=[layt[e[0]][0],layt[e[1]][0], None]# tọa độ theo trục x của đầu cạnh
  Ye+=[layt[e[0]][1],layt[e[1]][1], None]
  Ze+=[layt[e[0]][2],layt[e[1]][2], None]

Giải thích code bên trong vòng lặp ở trên

In [0]:
e = edges[0]
print('e: ',e)
print('layt[e[0]]: ',layt[e[0]])
print('layt[e[0]][0]: ',layt[e[0]][0])
print('layt[e[1]]: ',layt[e[1]])
print('layt[e[1]][0]: ',layt[e[1]][0])

e:  (0, 819)
layt[e[0]]:  [-14.419033909554377, 3.5502182186181614, -17.325026103948023]
layt[e[0]][0]:  -14.419033909554377
layt[e[1]]:  [-13.569102650133628, 2.9503275700308884, -18.197894052971463]
layt[e[1]][0]:  -13.569102650133628


In [0]:
print(len(Xn))
print(Xn[0])

1441
-14.419033909554377


In [0]:
# Tọa độ 2 đầu cạnh theo chiều x
print(Xe[:3])
print(len(Xe))

[-14.419033909554377, -13.569102650133628, None]
67182


67183 / 3 = 22394 edges

Chuyển qua tên để vẽ đồ thị

In [0]:
info = []
for i in range(len(nodes)):
  uid = nodes[i]
  name = dict_user_uid[uid]
  info.append(nodes[i]+'<br>'+name)
print(info[0])
# Output: 100010630570591<br>Trần Công

100010630570591<br>Trần Công


In [0]:
print(nodes[1])
print(info[1])

100003265219762
100003265219762<br>Lắc Si Lê


Xác định màu cho các node

In [0]:
color = []
for item in array_2D:
  len_item = len(item)
  if(len_item == 1):
    color.append(10)
  elif(len_item == 2):
    color.append(8)
  elif(len_item == 3):
    color.append(6)
  elif(len_item < 5):
    color.append(4)
  elif(len_item < 10):
    color.append(3)
  elif(len_item < 50):
    color.append(2)
  else:
    color.append(1)

In [0]:
from plotly.offline import plot
import plotly.graph_objs as go
#Thêm cạnh
trace1=go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=dict(color='rgb(125,125,125)', width=1),
               hoverinfo='none'
               )
#Thêm node
trace2=go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               marker=dict(symbol='circle',
                             size=5,
                             color=color,
                             line=dict(color='rgb(50,50,50)', width=0.3),
                             ),
               text=info,
               hoverinfo='text'
               )

axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

layout = go.Layout(
         title="Network of friends in the group UIT K11 (3D visualization)",
         width=1366,
         height=768,
         showlegend=False,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
        ),
    hovermode= 'closest'
         )

In [0]:
data=[trace1, trace2]
fig=go.Figure(data=data, layout=layout)
plot(fig, filename='3d_network_graphs.html')

'3d_network_graphs.html'

Toàn bộ source code tại https://github.com/duyquoc1508/Social-Network-Analysis.git